In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
/kaggle/input/mobilenet-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5
/kaggle/input/oxford-102-flowers/102flowers.tgz
/kaggle/input/oxford-102-flowers/imagelabels.mat


In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import zipfile
import seaborn as sns
import tensorflow as tf
#from google.colab.patches import cv2_imshow
#from google.colab import drive
%matplotlib inline
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense

In [5]:
import tarfile         # To deal with tgz file
path='/kaggle/input/oxford-102-flowers/102flowers.tgz'
tfile = tarfile.open(path, "r:gz")
tfile.extractall('./')
tfile.close()

In [6]:
import scipy.io      #To deal with mat file
mat = scipy.io.loadmat('/kaggle/input/oxford-102-flowers/imagelabels.mat')
label=mat["labels"].tolist()
l=np.reshape(label,(-1,1))

In [7]:
directory='/kaggle/working/jpg'
files=[os.path.join(directory,f) for f in sorted(os.listdir(directory))]

In [8]:
df=pd.DataFrame(columns=["Files","labels"])        #
df["Files"]=files
df["labels"]=l
y=df["labels"]
df.head()

,Files,labels
0,/kaggle/working/jpg/image_00001.jpg,77
1,/kaggle/working/jpg/image_00002.jpg,77
2,/kaggle/working/jpg/image_00003.jpg,77
3,/kaggle/working/jpg/image_00004.jpg,77
4,/kaggle/working/jpg/image_00005.jpg,77


In [9]:
output_shape = (128,128)
images=[]
for image_path in files :
  img = Image.open(image_path)
  img_resized = np.array(img.resize(output_shape, Image.ANTIALIAS))
  images.append(img_resized)


/tmp/ipykernel_47/2848993762.py:5: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = np.array(img.resize(output_shape, Image.ANTIALIAS))


In [10]:
images[0].shape

(128, 128, 3)

In [11]:
normalized_images = []
for img in images:
    # Convert the image to float32 for normalization
    img_float = img.astype(np.float32)

    # Normalize the pixel values using cv2.NORM_MINMAX
    img_normalized = cv2.normalize(img_float, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Append the normalized image to the list
    normalized_images.append(img_normalized)

In [12]:
x=np.array(normalized_images)
Y=np.array(y)
X_train,X_test,y_train,y_test=train_test_split(x,Y,test_size=0.2,random_state=42,shuffle=True)
X_trainn,X_val,y_trainn,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=42,shuffle=True)
X_trainn.shape,y_trainn.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape

((5240, 128, 128, 3),
 (5240,),
 (1311, 128, 128, 3),
 (1311,),
 (1638, 128, 128, 3),
 (1638,))

In [13]:
from keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
#weights="/kaggle/input/weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
from tensorflow.keras.applications import MobileNetV2

In [15]:
input_shape=(128,128,3)
weights="/kaggle/input/mobilenet-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
inp = Input(shape=input_shape)
mobile_model = MobileNetV2(weights=None,include_top=False,input_tensor=inp)
mobile_model.load_weights(weights)
mobile_model.trainable = False
mobile_model.summary()

Model: "mobilenetv2_1.00_None"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_3[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1

In [17]:
x= mobile_model.output
x = Flatten()(x)
x = Dense(10291, activation='relu')(x)
predictions = Dense(103, activation='softmax')(x)
model = Model(inputs=inp, outputs=predictions)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_3[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[0][0]']      

In [18]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_trainn, y_trainn, epochs=10, batch_size=20,validation_data=(X_val,y_val),callbacks=[early_stopping])

Epoch 1/10
262/262 [==============================] - 613s 2s/step - loss: 6.9244 - accuracy: 0.5227 - val_loss: 1.3090 - val_accuracy: 0.6613
Epoch 2/10
262/262 [==============================] - 670s 3s/step - loss: 0.6197 - accuracy: 0.8288 - val_loss: 1.3108 - val_accuracy: 0.7201
Epoch 3/10
262/262 [==============================] - 627s 2s/step - loss: 0.3578 - accuracy: 0.9008 - val_loss: 1.4702 - val_accuracy: 0.7132
Epoch 4/10
262/262 [==============================] - 630s 2s/step - loss: 0.2562 - accuracy: 0.9349 - val_loss: 1.7382 - val_accuracy: 0.6705
Epoch 5/10
262/262 [==============================] - 666s 3s/step - loss: 0.3752 - accuracy: 0.9143 - val_loss: 2.1553 - val_accuracy: 0.6804
Epoch 6/10
262/262 [==============================] - 640s 2s/step - loss: 0.2567 - accuracy: 0.9370 - val_loss: 2.1232 - val_accuracy: 0.7132


In [19]:
predictions=model.predict(X_test)
predictions

52/52 [==============================] - 20s 362ms/step


array([[1.7500632e-09, 2.2883358e-04, 1.5180497e-04, ..., 2.4833823e-06,
        1.4873993e-04, 2.4533934e-05],
       [1.8432571e-08, 2.1208653e-03, 1.0100544e-04, ..., 2.2149442e-03,
        9.4825173e-06, 4.6110186e-03],
       [7.6281749e-13, 1.9407906e-07, 1.1954306e-04, ..., 6.9330065e-07,
        5.9477948e-08, 1.2696914e-04],
       ...,
       [1.4698753e-07, 1.2997473e-03, 3.0363127e-04, ..., 6.7698694e-04,
        5.8371606e-06, 4.8140646e-05],
       [9.0228075e-11, 3.3517327e-07, 6.5012434e-10, ..., 1.4163017e-09,
        2.9678281e-11, 5.9606220e-10],
       [1.0548332e-11, 1.7459112e-09, 1.9868561e-04, ..., 2.8441699e-10,
        3.0934566e-06, 2.0499526e-08]], dtype=float32)

In [20]:
predictions=np.argmax(predictions,axis=1)
predictions

array([75, 73, 91, ..., 78, 30, 40])

In [21]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_test,predictions)

0.6642246642246642